# Итераторы

## Порядок сдачи домашнего

Под каждое домашнее вы создаете отдельную ветку куда вносите все изменения в рамках домашнего. Как только домашнее готово - создаете пулл реквест (обратите внимание что в пулл реквесте должны быть отражены все изменения в рамках домашнего). Ревьювера назначаете из таблицы - https://docs.google.com/spreadsheets/d/1vK6IgEqaqXniUJAQOOspiL_tx3EYTSXW1cUrMHAZFr8/edit?gid=0#gid=0
Перед сдачей проверьте код, напишите тесты. Не забудьте про PEP8, например, с помощью flake8. Задание нужно делать в jupyter notebook.

**Дедлайн - 11 ноября 10:00**

## Итератор по цифрам

Реализуйте класс-итератор `DigitIterator`, который принимает на вход целое число и позволяет итерироваться по его цифрам слева направо. На каждой итерации должна возвращаться следующая цифра числа.

**Условия:**
1.	Число может быть как положительным, так и отрицательным.
2.	Итератор должен возвращать только цифры числа, без знака - для отрицательных чисел.
3.	Итерация должна быть возможна с помощью цикла for или функции next().

**Пример использования:**

```python
iterator = DigitIterator(12345)
for digit in iterator:
    print(digit)
# 1
# 2
# 3
# 4
# 5

iterator = DigitIterator(-6789)
for digit in iterator:
    print(digit)

# 6
# 7
# 8
# 9
```

In [1]:
class DigitIterator:
    def __init__(self, number):
        self.digits = str(abs(number))
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.index < len(self.digits):
            digit = int(self.digits[self.index])
            self.index += 1
            return digit
        else:
            raise StopIteration

# Пример использования
iterator = DigitIterator(12345)
for digit in iterator:
    print(digit)

1
2
3
4
5


In [2]:
import unittest

class TestDigitIterator(unittest.TestCase):
    def test_positive_number(self):
        iterator = DigitIterator(12345)
        result = [digit for digit in iterator]
        self.assertEqual(result, [1, 2, 3, 4, 5])

    def test_negative_number(self):
        iterator = DigitIterator(-6789)
        result = [digit for digit in iterator]
        self.assertEqual(result, [6, 7, 8, 9])

    def test_single_digit(self):
        iterator = DigitIterator(5)
        result = [digit for digit in iterator]
        self.assertEqual(result, [5])

    def test_zero(self):
        iterator = DigitIterator(0)
        result = [digit for digit in iterator]
        self.assertEqual(result, [0])

# Итератор по файлу чанками

Реализуйте класс-итератор `FileChunkIterator`, который принимает на вход путь к файлу и количество байт для чтения. Итератор должен открывать файл и считывать его содержимое блоками фиксированного размера (количества байт), переданного в качестве параметра. При каждой итерации возвращается следующий блок байт, пока не будет достигнут конец файла.

**Условия:**
1.	Итератор должен открывать файл в режиме чтения бинарных данных (rb).
2.	Размер блока (количество байт) передаётся при создании итератора.
3.	Если в конце файла остаётся блок меньшего размера, итератор должен вернуть оставшиеся байты.
4.	При достижении конца файла итератор должен завершить работу, поднимая StopIteration.

**Пример использования:**
```python
with open("example.txt", "w") as file:
    file.write("Hello world!!")
    
iterator = FileChunkIterator("example.txt", 2)
for chunk in iterator:
    print(chunk)
# He
# ll
# o 
# wo
# rl
# d!
# !
```

In [3]:
class FileChunkIterator:
    def __init__(self, file_path, chunk_size):
        self.file_path = file_path
        self.chunk_size = chunk_size
        self.file = None

    def __iter__(self):
        self.file = open(self.file_path, "rb")
        return self

    def __next__(self):
        if self.file is None:
            raise StopIteration
        
        chunk = self.file.read(self.chunk_size)
        if chunk:
            return chunk
        else:
            self.file.close()
            raise StopIteration

    def __del__(self):
        if self.file and not self.file.closed:
            self.file.close()

In [4]:
with open("example.txt", "w") as file:
    file.write("Hello world!!")

iterator = FileChunkIterator("example.txt", 2)
for chunk in iterator:
    print(chunk.decode('utf-8'))

He
ll
o 
wo
rl
d!
!


In [15]:
import tempfile

class TestFileChunkIterator(unittest.TestCase):
    def setUp(self):
        self.temp_file = tempfile.NamedTemporaryFile(delete=False)
        self.temp_file.write(b"Hello world!!")
        self.temp_file.close()

    def tearDown(self):
        os.remove(self.temp_file.name)

    def test_chunk_size_2(self):
        iterator = FileChunkIterator(self.temp_file.name, 2)
        result = [chunk.decode('utf-8') for chunk in iterator]
        self.assertEqual(result, ["He", "ll", "o ", "wo", "rl", "d!", "!"])

    def test_chunk_size_5(self):
        iterator = FileChunkIterator(self.temp_file.name, 5)
        result = [chunk.decode('utf-8') for chunk in iterator]
        self.assertEqual(result, ["Hello", " worl", "d!!"])

    def test_large_chunk_size(self):
        iterator = FileChunkIterator(self.temp_file.name, 20)
        result = [chunk.decode('utf-8') for chunk in iterator]
        self.assertEqual(result, ["Hello world!!"])

# Итератор по подматрицам

Реализуйте класс-итератор `SubmatrixIterator`, который принимает на вход матрицу и размер подматрицы (квадратного блока). Итератор должен проходить по всем возможным подматрицам указанного размера и возвращать их одну за другой.

**Пример использования:**

```python
matrix = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]
]
iterator = SubmatrixIterator(matrix, 2)
for submatrix in iterator:
    print(submatrix)
    
# [[1, 2], [5, 6]]
# [[2, 3], [6, 7]]
# [[3, 4], [7, 8]]
# [[5, 6], [9, 10]]
# [[6, 7], [10, 11]]
# [[7, 8], [11, 12]]
# [[9, 10], [13, 14]]
# [[10, 11], [14, 15]]
# [[11, 12], [15, 16]]
```

In [6]:
class SubmatrixIterator:
    def __init__(self, matrix, submatrix_size):
        self.matrix = matrix
        self.submatrix_size = submatrix_size
        self.rows = len(matrix)
        self.cols = len(matrix[0]) if matrix else 0
        self.current_row = 0
        self.current_col = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.current_row > self.rows - self.submatrix_size:
            raise StopIteration

        submatrix = [
            self.matrix[i][self.current_col:self.current_col + self.submatrix_size]
            for i in range(self.current_row, self.current_row + self.submatrix_size)
        ]

        self.current_col += 1
        if self.current_col > self.cols - self.submatrix_size:
            self.current_col = 0
            self.current_row += 1

        return submatrix

In [7]:
matrix = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]
]
iterator = SubmatrixIterator(matrix, 2)
for submatrix in iterator:
    print(submatrix)

[[1, 2], [5, 6]]
[[2, 3], [6, 7]]
[[3, 4], [7, 8]]
[[5, 6], [9, 10]]
[[6, 7], [10, 11]]
[[7, 8], [11, 12]]
[[9, 10], [13, 14]]
[[10, 11], [14, 15]]
[[11, 12], [15, 16]]


In [8]:
class TestSubmatrixIterator(unittest.TestCase):
    def test_2x2_submatrix(self):
        matrix = [
            [1, 2, 3, 4],
            [5, 6, 7, 8],
            [9, 10, 11, 12],
            [13, 14, 15, 16]
        ]
        iterator = SubmatrixIterator(matrix, 2)
        result = [submatrix for submatrix in iterator]
        expected = [
            [[1, 2], [5, 6]],
            [[2, 3], [6, 7]],
            [[3, 4], [7, 8]],
            [[5, 6], [9, 10]],
            [[6, 7], [10, 11]],
            [[7, 8], [11, 12]],
            [[9, 10], [13, 14]],
            [[10, 11], [14, 15]],
            [[11, 12], [15, 16]]
        ]
        self.assertEqual(result, expected)

    def test_3x3_submatrix(self):
        matrix = [
            [1, 2, 3, 4],
            [5, 6, 7, 8],
            [9, 10, 11, 12],
            [13, 14, 15, 16]
        ]
        iterator = SubmatrixIterator(matrix, 3)
        result = [submatrix for submatrix in iterator]
        expected = [
            [[1, 2, 3], [5, 6, 7], [9, 10, 11]],
            [[2, 3, 4], [6, 7, 8], [10, 11, 12]],
            [[5, 6, 7], [9, 10, 11], [13, 14, 15]],
            [[6, 7, 8], [10, 11, 12], [14, 15, 16]]
        ]
        self.assertEqual(result, expected)

# Построчного чтение всех файлов в директории

Реализуйте класс-итератор  `RecursiveFileLineIteratorNoHidden`, который принимает на вход путь к директории и рекурсивно проходит по всем файлам, включая файлы во вложенных директориях. Итератор должен возвращать строки из каждого файла построчно, игнорируя файлы и директории, названия которых начинаются с точки (.), т.е. скрытые файлы и папки.

**Условия:**
1.	Итератор должен проходить по всем файлам в указанной директории и всех её поддиректориях, кроме тех, что начинаются с точки (.).
2.	Итератор должен возвращать строки из каждого файла поочерёдно, построчно.
3.	Поддерживаются только текстовые файлы.
4.	После завершения чтения всех файлов итератор должен завершить работу, поднимая StopIteration.
5.	Обработайте ситуацию, если файл не может быть открыт (например, из-за ошибок доступа).

**Пример использования:**

```python
iterator = RecursiveFileLineIteratorNoHidden("./test")
for line in iterator:
    print(line)
    
# Example 1
# Example 2
# Example 3
# Example 4
# Subfolder Example 1
# Subfolder Example 2
# Subfolder Example 3
# Subfolder Example 4    
```

Для выполнения задания потребуются несколько методов из модуля os, которые позволяют работать с файловой системой в Python. Давайте подробно рассмотрим их.


`os.walk(top, topdown=True, onerror=None, followlinks=False)` — это генератор, который рекурсивно обходит директории и поддиректории, начиная с указанного пути top. На каждом шаге возвращается кортеж, содержащий текущую директорию, список поддиректорий и список файлов.

Возвращаемые значения:
* root: Текущая директория, в которой находимся в данный момент обхода.
* dirs: Список поддиректорий в текущей root директории.
* files: Список файлов в текущей root директории.

`os.path.join(path, *paths)` объединяет один или несколько компонентов пути, возвращая корректный путь, соответствующий операционной системе. Это полезно для построения путей к файлам и директориям в кросс-платформенном формате.

```python
root = "/path/to/directory"
file_name = "example.txt"
full_path = os.path.join(root, file_name)
print(full_path)  # Вывод: "/path/to/directory/example.txt"
```

`os.path.isfile(path)` проверяет, является ли указанный путь файлом. Возвращает True, если path указывает на файл, и False, если это директория или объект другого типа.

```python
file_path = "/path/to/file.txt"
if os.path.isfile(file_path):
    print("Это файл.")
else:
    print("Это не файл.")
```

`os.path.basename(path)` возвращает базовое имя файла или директории из пути. Это полезно, если нужно получить только имя файла или папки, без остальных компонентов пути.

```python
file_path = "/path/to/file.txt"
print(os.path.basename(file_path))  # Вывод: "file.txt"
```

`os.path.isdir(path)` проверяет, является ли указанный путь директорией. Возвращает True, если path указывает на директорию, и False, если это файл или объект другого типа.

```python
dir_path = "/path/to/directory"
if os.path.isdir(dir_path):
    print("Это директория.")
else:
    print("Это не директория.")
```

In [9]:
import os

class RecursiveFileLineIteratorNoHidden:
    def __init__(self, directory):
        self.directory = directory
        self.files = []
        self.file_index = 0
        self.current_file = None

        for root, dirs, files in os.walk(self.directory):
            dirs[:] = [d for d in dirs if not d.startswith('.')]
            for file in files:
                if not file.startswith('.'):
                    self.files.append(os.path.join(root, file))

    def __iter__(self):
        return self

    def __next__(self):
        while self.current_file is None or self.current_line is None:
            if self.file_index >= len(self.files):
                raise StopIteration

            file_path = self.files[self.file_index]
            self.file_index += 1

            try:
                self.current_file = open(file_path, 'r', encoding='utf-8')
                self.current_line = self.current_file.readline()
            except (IOError, OSError):
                self.current_file = None
                continue

        line = self.current_line
        self.current_line = self.current_file.readline()

        if not self.current_line:
            self.current_file.close()
            self.current_file = None
            self.current_line = None

        return line.strip()

    def __del__(self):
        if self.current_file and not self.current_file.closed:
            self.current_file.close()

In [10]:
iterator = RecursiveFileLineIteratorNoHidden("./test")
for line in iterator:
    print(line)

Example 1
Example 2
Example 3
Example 4
Subfolder Example 1
Subfolder Example 2
Subfolder Example 3
Subfolder Example 4


In [16]:
class TestRecursiveFileLineIteratorNoHidden(unittest.TestCase):
    def setUp(self):
        self.test_dir = tempfile.TemporaryDirectory()
        with open(os.path.join(self.test_dir.name, "file1.txt"), "w") as f:
            f.write("Hello\nworld!!\n")
        os.makedirs(os.path.join(self.test_dir.name, "subfolder"))
        with open(os.path.join(self.test_dir.name, "subfolder", "file2.txt"), "w") as f:
            f.write("Subfolder Example 1\nSubfolder Example 2\n")

    def tearDown(self):
        self.test_dir.cleanup()

    def test_line_iteration(self):
        iterator = RecursiveFileLineIteratorNoHidden(self.test_dir.name)
        result = [line for line in iterator]
        expected = [
            "Hello",
            "world!!",
            "Subfolder Example 1",
            "Subfolder Example 2"
        ]
        self.assertEqual(result, expected)

    def test_hidden_files_ignored(self):
        with open(os.path.join(self.test_dir.name, ".hidden_file.txt"), "w") as f:
            f.write("This should be ignored.\n")
        iterator = RecursiveFileLineIteratorNoHidden(self.test_dir.name)
        result = [line for line in iterator]
        expected = [
            "Hello",
            "world!!",
            "Subfolder Example 1",
            "Subfolder Example 2"
        ]
        self.assertEqual(result, expected)

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)

...........
----------------------------------------------------------------------
Ran 11 tests in 0.024s

OK
